In [1]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [2]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [3]:
data.loc[data['2030val'].apply(pd.isna), '2030val'] = 0
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [4]:
data_gid = data[['gid', 'geometry']]

In [5]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."



## 강남구서초구 출근 자전거

In [19]:
with open('output/그리드 전처리 완료/자전거이용/강남서초출근자전거교차2 - 복사본.json',encoding='UTF-8') as json_file:
    json_data = json.load(json_file)

In [124]:
전처리전 = json_data["features"]

In [ ]:
'properties': {'COUNT': 0, 'gid': '다사605367'},


In [125]:
json_data["features"][0]["properties"]

{'COUNT': 0, 'gid': '다사605367'}

In [131]:
json_data["features"][16546]["properties"]

{'COUNT': 64, 'gid': '다사590486'}

In [137]:
countgid = []
for i in range(0,16547) :
    a = json_data["features"][i]["properties"]
    countgid.append(a)
    
countgid

[{'COUNT': 0, 'gid': '다사605367'},
 {'COUNT': 0, 'gid': '다사606367'},
 {'COUNT': 0, 'gid': '다사604366'},
 {'COUNT': 0, 'gid': '다사603366'},
 {'COUNT': 0, 'gid': '다사604367'},
 {'COUNT': 0, 'gid': '다사601369'},
 {'COUNT': 0, 'gid': '다사602368'},
 {'COUNT': 0, 'gid': '다사602369'},
 {'COUNT': 0, 'gid': '다사602367'},
 {'COUNT': 0, 'gid': '다사603369'},
 {'COUNT': 0, 'gid': '다사604368'},
 {'COUNT': 0, 'gid': '다사604366'},
 {'COUNT': 0, 'gid': '다사603366'},
 {'COUNT': 0, 'gid': '다사603368'},
 {'COUNT': 0, 'gid': '다사603367'},
 {'COUNT': 0, 'gid': '다사600368'},
 {'COUNT': 0, 'gid': '다사604367'},
 {'COUNT': 0, 'gid': '다사601368'},
 {'COUNT': 0, 'gid': '다사607369'},
 {'COUNT': 0, 'gid': '다사609369'},
 {'COUNT': 0, 'gid': '다사606367'},
 {'COUNT': 0, 'gid': '다사610368'},
 {'COUNT': 0, 'gid': '다사606368'},
 {'COUNT': 0, 'gid': '다사609368'},
 {'COUNT': 0, 'gid': '다사608369'},
 {'COUNT': 0, 'gid': '다사607368'},
 {'COUNT': 0, 'gid': '다사607367'},
 {'COUNT': 0, 'gid': '다사608368'},
 {'COUNT': 0, 'gid': '다사614367'},
 {'COUNT': 0, 

In [138]:
df = pd.DataFrame(countgid)
df

,COUNT,gid
0,0,다사605367
1,0,다사606367
2,0,다사604366
3,0,다사603366
4,0,다사604367
...,...,...
16542,22,다사586486
16543,22,다사585486
16544,64,다사592486
16545,64,다사591486


In [142]:
result1 = df[['COUNT', 'gid']].groupby(['gid']).sum()
result1

,COUNT
gid,
다사540449,38
다사540450,150
다사540451,81
다사540452,81
다사540453,81
...,...
다사666409,7
다사666410,5
다사666411,10


In [143]:
result1.reset_index(inplace=True)

In [144]:
result1

,gid,COUNT
0,다사540449,38
1,다사540450,150
2,다사540451,81
3,다사540452,81
4,다사540453,81
...,...,...
8937,다사666409,7
8938,다사666410,5
8939,다사666411,10
8940,다사667408,5


In [145]:
result1.to_csv("output/그리드 전처리 완료/서초강남출근시간자전거.csv")

## 강남구서초구 퇴근 자전거

In [132]:
with open('output/그리드 전처리 완료/자전거이용/강남서초퇴근자전거교차2 - 복사본.json',encoding='UTF-8') as json_file:
    json_data2 = json.load(json_file)

In [ ]:
json_data2

In [140]:
countgid2 = []
for i in range(0,16547) :
    a = json_data2["features"][i]["properties"]
    countgid2.append(a)
    
countgid2

[{'COUNT': 0, 'gid': '다사605367'},
 {'COUNT': 0, 'gid': '다사606367'},
 {'COUNT': 0, 'gid': '다사604366'},
 {'COUNT': 0, 'gid': '다사603366'},
 {'COUNT': 0, 'gid': '다사604367'},
 {'COUNT': 0, 'gid': '다사601369'},
 {'COUNT': 0, 'gid': '다사602368'},
 {'COUNT': 0, 'gid': '다사602369'},
 {'COUNT': 0, 'gid': '다사602367'},
 {'COUNT': 0, 'gid': '다사603369'},
 {'COUNT': 0, 'gid': '다사604368'},
 {'COUNT': 0, 'gid': '다사604366'},
 {'COUNT': 0, 'gid': '다사603366'},
 {'COUNT': 0, 'gid': '다사603368'},
 {'COUNT': 0, 'gid': '다사603367'},
 {'COUNT': 0, 'gid': '다사600368'},
 {'COUNT': 0, 'gid': '다사604367'},
 {'COUNT': 0, 'gid': '다사601368'},
 {'COUNT': 0, 'gid': '다사607369'},
 {'COUNT': 0, 'gid': '다사609369'},
 {'COUNT': 0, 'gid': '다사606367'},
 {'COUNT': 0, 'gid': '다사610368'},
 {'COUNT': 0, 'gid': '다사606368'},
 {'COUNT': 0, 'gid': '다사609368'},
 {'COUNT': 0, 'gid': '다사608369'},
 {'COUNT': 0, 'gid': '다사607368'},
 {'COUNT': 0, 'gid': '다사607367'},
 {'COUNT': 0, 'gid': '다사608368'},
 {'COUNT': 0, 'gid': '다사614367'},
 {'COUNT': 0, 

In [141]:
df2 = pd.DataFrame(countgid2)
df2

,COUNT,gid
0,0,다사605367
1,0,다사606367
2,0,다사604366
3,0,다사603366
4,0,다사604367
...,...,...
16542,135,다사586486
16543,135,다사585486
16544,416,다사592486
16545,416,다사591486


In [146]:
result2 = df2[['COUNT', 'gid']].groupby(['gid']).sum()
result2

,COUNT
gid,
다사540449,217
다사540450,951
다사540451,530
다사540452,530
다사540453,530
...,...
다사666409,8
다사666410,7
다사666411,21


In [147]:
result2.reset_index(inplace=True)

In [148]:
result2

,gid,COUNT
0,다사540449,217
1,다사540450,951
2,다사540451,530
3,다사540452,530
4,다사540453,530
...,...,...
8937,다사666409,8
8938,다사666410,7
8939,다사666411,21
8940,다사667408,4


In [149]:
result2.to_csv("output/그리드 전처리 완료/서초강남퇴근시간자전거.csv")